In [2]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd

import json
import pymongo as pm

## purpose

* upload sketches to S3 [maybe do this later]
* build stimulus dictionary and write to database

## upload sketches to S3 [todo later]

## build stimulus dictionary

In [3]:
## read in experimental metadata file (CSV)
path_to_metadata = 'sketchpad_basic_group_data_for_annotation.csv'
meta = pd.read_csv(path_to_metadata)    

In [33]:
## add parts list
parts =[]
for i in range(meta.shape[0]-1):
    if meta.category[i]=="chair":
        parts.append(["leg","armrest","backrest","seat","pedestal","wheel"])
    if meta.category[i]=="dog":
        parts.append(["eye","mouth","ear","body","tail","neck","leg"])
    if meta.category[i] == "bird":
        parts.append(["beak","wings","feet","tail","body","head"])
    if meta.category[i] == "car":
        parts.append(["wheel", "windshield", "body", "window", "door", "bumper", "lamp"])
##parts = pd.Series(parts)
meta = meta.assign(parts=pd.Series(parts))
meta

,gameID,trialNum,condition,target,drawDuration,outcome,response,numStrokes,Distractor1,Distractor2,Distractor3,pose,iteration,svg,viewerRT,mean_intensity,category,parts
0,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,1,further,knob,18.389,True,knob,7,crow,redsport,basset,35,pilot2,"[u'M69,167c0,27.13435 5,54.84972 5,81c0,1.8150...",6.656,0.0347,chair,"[leg, armrest, backrest, seat, pedestal, wheel]"
1,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,2,further,redsport,2.820,True,redsport,3,crow,knob,basset,35,pilot2,"[u'M51,224c0,-65.25556 75.24654,-35.24654 83,-...",5.607,0.0320,car,"[wheel, windshield, body, window, door, bumper..."
2,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,3,closer,bluesport,22.093,False,beetle,5,beetle,hatchback,bluesedan,35,pilot2,"[u'M34,212c5.3353,-10.6706 6.56215,-22.81075 9...",6.695,0.0466,car,"[wheel, windshield, body, window, door, bumper..."
3,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,4,closer,chihuahua,24.122,True,chihuahua,14,weimaraner,doberman,pug,35,pilot2,"[u'M99,157c-26.4479,17.63193 -2.9121,54.60807 ...",3.133,0.0371,dog,"[eye, mouth, ear, body, tail, neck, leg]"
4,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,5,closer,pug,34.260,False,chihuahua,12,weimaraner,doberman,chihuahua,35,pilot2,"[u'M69,172c0,4.72338 -5.37528,22.62472 -2,26c9...",5.739,0.0381,dog,"[eye, mouth, ear, body, tail, neck, leg]"
5,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,6,closer,weimaraner,27.287,False,doberman,15,doberman,chihuahua,pug,35,pilot2,"[u'M31,149c31.51565,0 63.69848,1.39154 95,4c27...",2.241,0.0496,dog,"[eye, mouth, ear, body, tail, neck, leg]"
6,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,7,further,redantique,4.392,True,redantique,4,robin,sling,bullmastiff,35,pilot2,"[u'M39,203c-1.61473,-12.9178 -11.98219,-32.508...",2.127,0.0390,car,"[wheel, windshield, body, window, door, bumper..."
7,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,8,further,basset,10.582,True,basset,8,crow,redsport,knob,35,pilot2,"[u'M136,123c-10.03489,0 -22.63418,-2.12194 -32...",3.965,0.0453,dog,"[eye, mouth, ear, body, tail, neck, leg]"
8,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,9,closer,waiting,21.246,True,waiting,11,leather,inlay,squat,35,pilot2,"[u'M65,288c0,-47.26597 -10.61179,-88.33968 29,...",1.793,0.0722,chair,"[leg, armrest, backrest, seat, pedestal, wheel]"
9,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,10,closer,inlay,25.374,True,inlay,14,leather,waiting,squat,35,pilot2,"[u'M53,198c5.4241,-32.54462 25.86878,-21.08688...",1.847,0.0566,chair,"[leg, armrest, backrest, seat, pedestal, wheel]"


In [4]:
## add iteration name information
_iterationName = 'sketchpad_basic'
iterationName = [_iterationName]*len(meta)
meta = meta.assign(iterationName=pd.Series(iterationName))

In [5]:
## svg string formatting
meta['svg'] = meta['svg'].str.replace('u', '')
meta['svg'] = meta['svg'].str.replace("'",'"') 

In [6]:
## add empty games list
games = [[] for i in np.arange(len(meta))]
meta = meta.assign(games=pd.Series(games))

In [7]:
## how many games worth of data do we have?
print '{} unique games worth of data.'.format(len(np.unique(meta.gameID.values)))
print '{} unique sketches.'.format(len(meta))

## write out metadata to json file

## for example:
stimdict = meta.to_dict(orient='records')
stimdict
import json
with open('annotation_meta.js', 'w') as fout:
     json.dump(stimdict, fout)

93 unique games worth of data.
2976 unique sketches.


### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [8]:
## load in the JSON that contains the svgData, object labels, and part labels
J = json.loads(open('annotation_meta.js',mode='rU').read())
assert len(J)==len(meta)

In [9]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [10]:
## define the dbname and collection name
db = conn['stimuli']
coll = db['svg_annotation_sketchpad_basic']

In [11]:
## actually add data now to the database (iff reallyRun==True)
reallyRun = False
if reallyRun:
    for (i,j) in enumerate(J):
        if i%250==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)
reallyRun = False        

0 of 2976
250 of 2976
500 of 2976
750 of 2976
1000 of 2976
1250 of 2976
1500 of 2976
1750 of 2976
2000 of 2976
2250 of 2976
2500 of 2976
2750 of 2976


In [12]:
## How many sketches do we have in the database?
print 'We have {} sketches.'.format(coll.count())

## What kind of sketches do we have in the database?
print 'We have these kinds: {}'.format(str(coll.distinct('category')))

We have 2976 sketches.
We have these kinds: [u'chair', u'car', u'dog', u'bird']
